In [ ]:
import numpy as np
import pandas as pd
import uproot
import awkward_pandas
import matplotlib.pyplot as plt
import os
import math
import subprocess
import time
import itertools

In [ ]:
def getGarfieldPath():
    """
    Reads the file "GARFIELDPATH" and returns the file path to setupGarfield.sh.
    This step is neccesary in order to initialize Garfield++ in the terminal before
    begining any simulations. The path currently needs to be manually set by the user
    within the "GARFIELDPATH" file. The later portion of the path should look like
    "/install/share/Garfield/setupGarfield.sh"
    """
    filename = 'GARFIELDPATH'
    try:
        with open(filename, 'r') as file:
            garfieldPath = file.read().strip()
            if not os.path.exists(garfieldPath):
                print(f"Error: File 'setupGarfield.sh' not found at {garfieldPath}.")
                return
            return garfieldPath

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

In [ ]:
def setupSimulation():
    """
    Initializez Garfield++ before running any simulations. This requires
    the path to setupGarfield.sh via the getGarfieldPath() command. This
    step could also be done via the bashrc in Linux or similar methods in 
    other operating systems.
    """
    #Get local Garfield path
    garfieldPath = getGarfieldPath()

    #Check for build/
    if not os.path.exists("build"):
        os.makedirs("build")
    
    #Make executable
    makeBuild = (
        f'source {garfieldPath} && '
        f'cd build && '
        f'cmake .. && '
        f'make'
    )
    result = subprocess.run(
        makeBuild,
        shell=True,
        check=True,
        executable='/bin/bash',
        capture_output=True,
        text=True
    )

    #Check for run number file
    if not os.path.exists("runNo"):
        with open(runNo, "w") as file:
            file.write('1000')
            
    return

In [ ]:
def defaultParam():
    """
    Sets all the adjustable parameters in the FIMS simulation to a
    default value. This includes the geometry, E-field strength, gas
    composition, and simulation limits. It returns the list of these
    new values.
    """
    defaultParam = {
        'padLength': 65.,
        'pitch': 250.,
        'gridStandoff': 100.,
        'gridThickness': .5,
        'holeRadius': 90.,
        'cathodeHeight': 200.,
        'thicknessSiO2': 5.,
        'fieldRatio': 40.,
        'numFieldLine': 200,
        'transparencyLimit': .1,
        'numAvalanche': 1000,
        'avalancheLimit': 200,
        'gasCompAr': 80.,
        'gasCompCO2': 20.,
    }
    return defaultParam

In [ ]:
def readParam():
    """
    Reads the "runControl" file and returns the variables in it as
    well as their values as a list. It is setup assuming that comments
    are made with // and each non-commented line terminates with ;.
    """
    filename = 'runControl'
    param = {}
    
    try:
        with open(filename, 'r') as file:
            for line in file:
                line = line.strip()  # Remove leading/trailing whitespace
                if line.startswith('/') or not line:  # Skip comments and empty lines
                    continue
                # Split the line at the '='
                parts = line.split('=', 1)
                if len(parts) == 2:
                    key, value = parts[0].strip(), parts[1].strip()
                    value = value.rstrip(';') # Remove trailing semicolon
                    param[key] = value
                else:
                    print(f"Skipping malformed line: {line}")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return {}
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return {}

    return param

In [ ]:
def writeRunControl(param=None):
    """
    Opens the "runControl" file and replaces the list of values in it
    with new values. The new values are taken as an input in the form
    of a list. The new values are written to the "runControl" file,
    so the command itself does not return anything.
    """
    filename = 'runControl'

    if param is None or not param:
        print(f"No parameters to write to {filename}.")
        return

    try:
        with open(filename, 'r') as file:
            oldLines = file.readlines()  # Read all lines of the file
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

    newLines = []
    for line in oldLines:
        line = line.strip()
        if line.startswith('/') or not line:
            newLines.append(line + '\n')  # Keep comments and empty lines
            continue

        parts = line.split('=', 1)
        if len(parts) == 2:
            key = parts[0].strip()
            if key in param:
                newLines.append(f"{key} = {param[key]};\n")  # Update value
            else:
                newLines.append(line + '\n') #keep original line
        else:
            newLines.append(line + '\n')  # Keep non-parameter lines

    try:
        with open(filename, 'w') as file:
            file.writelines(newLines)
    except Exception as e:
        print(f"An error occurred while writing to {filename}: {e}")
    
    return

In [ ]:
def calcPotentials(param=None):
    """
    Uses a list of parameters to calculate the voltages needed for 
    the Elmer .sif file. Returns the voltages as a list with the
    first element of the list being the cathode voltage and the
    second element being the grid voltage.
    """

    if param is None or not param:
        print(f"Invalid parameters.")
        return None
        
    convertEField = 0.1 # 1 kV/cm = 0.1 V/micron
    
    # Calculate the voltage required to achieve amplification field
    gridDistance = float(param['gridStandoff']) - float(param['gridThickness'])/2. #micron

    gridVoltage = float(param['fieldRatio'])*convertEField*gridDistance

    # Calculate for drift field
    cathodeDistance = float(param['cathodeHeight']) - float(param['gridThickness'])/2. #micron
    cathodeVoltage = convertEField*cathodeDistance + gridVoltage

    potentials = {
        'cathodeVoltage': -cathodeVoltage,
        'gridVoltage': -gridVoltage
    }
    
    return potentials

In [ ]:
def writeSIF(param=None):
    """
    Takes a list of parameters as an input and uses the calcPotentials()
    command to calculate the voltages needed for the Elmer .sif file.
    The list of voltages is then written to FIMS.sif, which is located
    in the Geometry folder.
    """
    filename = os.path.join('./Geometry', 'FIMS.sif')

    if param is None or not param:
        print(f"Invalid parameters.")
        return None
    
    try:
        with open(filename, 'r') as file:
            sifLines = file.readlines()  # Read all lines of the file

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

    potentials = calcPotentials(param)

    writeCathode = -1
    writeGrid = -1

    # Find the cathod and grid naming lines
    for i, line in enumerate(sifLines):
        if 'Name = "Cathode"' in line:
            writeCathode = i+1
        if 'Name = "Grid"' in line:
            writeGrid = i+1

    if writeCathode == -1 or 'Potential =' not in sifLines[writeCathode]:
        print('Error with cathode.')
        return
    if writeGrid == -1 or 'Potential =' not in sifLines[writeGrid]:
        print('Error with grid.')
        return
    
    sifLines[writeCathode] = f'\tPotential = {potentials['cathodeVoltage']}\n'
    sifLines[writeGrid] = f'\tPotential = {potentials['gridVoltage']}\n'
        
    try:
        with open(filename, 'w') as file:
            file.writelines(sifLines)
    except Exception as e:
        print(f"An error occurred while writing to {filename}: {e}")
        return
        
    return

In [ ]:
def writeParam(param=None):
    """
    Takes the list of parameters and then feeds them to the two
    write commands: writeRunControl() and writeSIF().
    """
    if param is None or not param:
        print(f"No parameters to write.")
        return    

    writeRunControl(param)
    writeSIF(param)
    
    return

In [ ]:
def resetParam(verbose=True):
    """
    Uses the defaaltParams() command to reset the list of
    params. It then sends the defaulted list to writeParam()
    so that the reset parameters are written to the "runControl"
    and "FIMS.sif" files. Finally, the defaulted parameters are
    returned as their own list.
    """
    defaultParams = defaultParam()
    writeParam(defaultParams)

    if verbose:
        print('Parameters have been reset.')
    
    return defaultParams

In [ ]:
def getRunNumber():
    """
    Reads the file "runNo" and returns the number given in the file.
    """
    filename = 'runNo'

    try:
        with open(filename, 'r') as file:
            content = file.read().strip()
            runNo = int(content)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except ValueError:
        print(f"Error: Invalid number format in '{filename}")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return
    
    return runNo

In [ ]:
def runGmsh():
    """
    Runs the gmsh external program from the command line. The location
    of gmsh may need to be adjusted depending on its location within
    the local system. Linux users will need to add "./" if gmsh is located
    in the current working directory. gmsh is run with order 2 to fit Elmer
    and Garfield++ element requirements. Elements sizes are determined by the
    smallest of the following: size given at a point, distance to a boundary, 
    or a default value determined by gmsh (currently unclear from gmsh 
    documentation).
    """
    
    with open(os.path.join(os.getcwd(), 'log/logGmsh.txt'), 'w+') as gmshOutput:
        startTime = time.monotonic()
        subprocess.run(['gmsh', os.path.join('./Geometry/', 'FIMS.txt'),
                        '-order', '2', 
                        '-optimize_ho',
                        '-clextend', '1',
                        '-setnumber', 'Mesh.OptimizeNetgen', '1',
                        '-setnumber', 'Mesh.MeshSizeFromPoints', '1',
                        '-3'],
                        stdout=gmshOutput)
        endTime = time.monotonic()
        gmshOutput.write(f'\n\nGmsh run time: {endTime - startTime} s')
    return

In [ ]:
def runElmer():
    """
    Runs the Elmer FEM external program from the command line. First, the command changes the current
    working directory to the Geometry folder (the same folder where FIMS.sif is located). It then
    converts the FIMS.msh file generated by the gmsh program into four seperate files that can be
    read by Elmer via the ElmerGrid command in the command line. ElmerSolver then generates the
    electric field based on the boundary conditions specified in FIMS.sif. The outputs of all
    Elmer commands are specified in the .sif file (currently /Geometry/elmerResults). The command
    then switches back to the original working directory.
    """
    
    originalCWD = os.getcwd()
    os.chdir('./Geometry')

    if not os.path.exists("elmerResults"):
        os.makedirs("elmerResults")
        
    try:
        with open(os.path.join(originalCWD, 'log/logElmerGrid.txt'), 'w+') as elmerOutput:
            startTime = time.monotonic()
            subprocess.run(['ElmerGrid', '14', '2', 'FIMS.msh', '-out', 'elmerResults', '-autoclean'], stdout=elmerOutput)
            endTime = time.monotonic()
            elmerOutput.write(f'\n\nElmerGrid run time: {endTime - startTime} s')
            
        with open(os.path.join(originalCWD, 'log/logElmerSolver.txt'), 'w+') as elmerOutput:
            startTime = time.monotonic()
            subprocess.run(['ElmerSolver', 'FIMS.sif'], stdout=elmerOutput)
            endTime = time.monotonic()
            elmerOutput.write(f'\n\nElmerSolver run time: {endTime - startTime} s')
    finally:
        os.chdir(originalCWD)
    return

In [ ]:
def runGarfield():
    """
    Makes and then runs the "runAvalanche" executable from the command line. First, the
    command changes the active directory to the "build" folder. It then uses the
    getGarfieldPath() command to initialize Garfield++ and then uses cmake to make the 
    "runAvalanche" executable. It then runs the executable before returning to the
    original working directory.
    """
    
    originalCWD = os.getcwd()
    garfieldPath = getGarfieldPath()
    os.chdir('./build/')
    try:
        with open(os.path.join(originalCWD, 'log/logGarfield.txt'), 'w+') as garfieldOutput:
            startTime = time.monotonic()
            setupAvalanche = (
                f'source {garfieldPath} && '
                f'make && '
                f'./runAvalanche'
            )
            subprocess.run(setupAvalanche, stdout=garfieldOutput, shell=True, check=True)
            endTime = time.monotonic()
            garfieldOutput.write(f'\n\nGarfield run time: {endTime - startTime} s')
    finally:
        os.chdir(originalCWD)
    return

In [ ]:
def runSimulation(param=None):
    """
    A wrapper function that uses the list of parameters to first update the "FIMS.txt"
    and "FIMS.sif" files, run the gmsh and Elmer external programs, and lastly execute
    the runGarfield() command.
    """

    if param is None or not param:
        print(f'No parameters to run.')
        return   

    print(f'Running simulation - Run number: {getRunNumber()}')
    writeParam(param)
    runGmsh()
    runElmer()
    runGarfield()
    return

In [ ]:
def varyParam(parameter, minVal, maxVal, numSteps=10):
    """
    This command takes four inputs: the parameter to be updated, the starting
    value of that parameter, the final value of the parameter, and the number
    of steps between the first and final value. The command begins by reseting 
    all parameters to their default values. It then finds the current run number 
    from "runNo" and updates it for the current simulation number. It then creates
    a linear space for the parameter being adjusted and runs the simulation via
    runSimulation() using the linear parameter space as a script for what values
    to use. It will return the linear space of the parameter and the updated 
    run number.
    """
    
    resetParam()
    param = readParam()
    
    if parameter not in param:
        print(f"Parameter '{parameter}' not able to be varied.")
        return None, None

    runNo = getRunNumber()
    print(f"Varying '{parameter}'. Initial run number: {runNo}")

    variedParam = np.linspace(minVal, maxVal, numSteps)
    runNos = [runNo + inRun for inRun in range(len(variedParam))]

    for inParam in variedParam:
        param[parameter] = inParam

        runSimulation(param)
            
    print(f"Done varying '{parameter}'.")

    param = resetParam()

    return variedParam, runNos

In [ ]:
def scanParam(parameters, minValues, maxValues, numSteps):
    """
     A function that takes four lists as an input. The first list contains
    the names (given as strings with quotes around each name) of the parameters
    that the user wishes to use. The second list contains the starting value for
    each parameter in the list. The third list contains final value for each
    paremeter in the list. The fourth and final list contains the number of
    steps between the first and final value for each parameter.
    
    The command begins by reseting all parameters to their default values. It then
    creates a linear space for the parameters being adjusted and prints the current 
    run number as well as the specified simulation details. It then runs the simulation
    via runSimulation() using the linear parameter space as a script for what values
    to use. The command will run n*m*...*l simulations, where n,m,...,l are the
    steps for each specified parameter. It then prints the final run number and returns
    a list of defaulted parameters.
    """

    if not (len(parameters) == len(minValues) == len(maxValues) == len(numSteps)):
        raise ValueError('Inputs must have the same length')

    resetParam()
    param = readParam()

    for inParam in parameters:
        if inParam not in param:
            print(f"Parameter '{inParam}' not able to be varied.")
            return None

    
    #Find range for each parameter
    allRanges = []
    for i, _ in enumerate(parameters):
        paramRange = np.linspace(minValues[i], maxValues[i], numSteps[i])
        allRanges.append(paramRange)

    print(f'Beginning scan through {parameters}. {math.prod(numSteps)} runs. Initial run: {getRunNumber()}')

    for scanParams in itertools.product(*allRanges):

        #Reset parameters to defaults
        param = resetParam(verbose=False)
        #Apply current parameter combination
        for i, inParam in enumerate(parameters):
            param[inParam] = scanParams[i]

        runSimulation(param)

    print(f'Done scan through {parameters}. Final run number: {getRunNumber()-1}')
    param = resetParam()
        
    return param

In [ ]:
def readRootTrees(runNumber: int) -> dict:
    """
    Reads all trees from a ROOT file and returns them as a dictionary.
    The keys are the tree names and the values are pandas DataFrames.

    Args:
        runNumber (int): The run number of the ROOT file.

    Returns:
        dict: A dictionary where keys are tree names (str) and
              values are pandas DataFrames. Returns an empty dictionary
              if the file cannot be opened or contains no trees.
    """

    dataFilePath = '/home/James/fims/DataGeneration/FIMS_SIM/Data/'
    dataFile = f'sim.{runNumber}.root'
    fullFilePath = os.path.join(dataFilePath, dataFile)
    
    try:
        with uproot.open(fullFilePath) as rootFile:
            dataframes = {}
            for treeName in rootFile.keys():
                if isinstance(rootFile[treeName], uproot.behaviors.TTree.TTree):
                    tree = rootFile[treeName]
                    try:
                        df = tree.arrays(library='pd')
                        dataframes[treeName] = df
                    except Exception as e:
                        print(f"Error reading tree '{treeName}': {e}")
            return dataframes
    except Exception as e:
        print(f"Error opening or reading ROOT file '{fullFilePath}': {e}")
        return {}

In [ ]:
def checkTransparency():
    """
    """
    
    runNum = getRunNumber() - 1
    simData = readRootTrees(runNum)
    transparency = simData['metaDataTree;1']['Field Transparency'][0]
    limit = simData['metaDataTree;1']['Field Transparency Limit'][0]
    
    if transparency < 1 - limit:
        return True
    
    return False

In [ ]:
def transparencyScan(parameters, minValues, maxValues, numSteps):
    """
     A function that takes four lists as an input. The first list contains
    the names (given as strings with quotes around each name) of the parameters
    that the user wishes to use. The second list contains the starting value for
    each parameter in the list. The third list contains final value for each
    paremeter in the list. The fourth and final list contains the number of
    steps between the first and final value for each parameter.
    
    The command begins by reseting all parameters to their default values. It then
    creates a linear space for the parameters being adjusted and prints the current 
    run number as well as the specified simulation details. It then runs the simulation
    via runSimulation() using the linear parameter space as a script for what values
    to use. The command will run n*m*...*l simulations, where n,m,...,l are the
    steps for each specified parameter. It then prints the final run number and returns
    a list of defaulted parameters.
    """

    if not (len(parameters) == len(minValues) == len(maxValues) == len(numSteps)):
        raise ValueError('Inputs must have the same length')

    resetParam()
    param = readParam()

    for inParam in parameters:
        if inParam not in param:
            print(f"Parameter '{inParam}' not able to be varied.")
            return None

    
    #Find range for each parameter
    allRanges = []
    for i, _ in enumerate(parameters):
        paramRange = np.linspace(minValues[i], maxValues[i], numSteps[i])
        allRanges.append(paramRange)

    print(f'Beginning scan through {parameters}. {math.prod(numSteps)} runs. Initial run: {getRunNumber()}')

    for scanParams in itertools.product(*allRanges):

        #Reset parameters to defaults
        param = resetParam(verbose=False)
        #Apply current parameter combination
        for i, inParam in enumerate(parameters):
            param[inParam] = scanParams[i]

        runSimulation(param)
        if checkTransparency():
            print('Field transparency too low. Ending scan')
            break

    print(f'Done scan through {parameters}. Final run number: {getRunNumber() - 1}')
    param = resetParam()
        
    return param

In [ ]:
#setupSimulation()
#scanParam(['holeRadius', 'gridStandoff', 'fieldRatio'],
#          [16, 80, 60],
#          [20, 120, 60],
#          [3, 3, 1])

In [ ]:
setupSimulation()
startTime = time.monotonic()
runSimulation(resetParam())
endTime = time.monotonic()
print(f'Test run takes ~{endTime-startTime:.1f}s.')